# Audio Data

## Introduction

Audio data - as recorded by smartphones or other portable devices - can carry important information about individuals' environments. This may give insights about the activity, sleep, and social interaction. However, using these data can be tricky due to privacy concerns, for example, conversations are highly identifiable. A possible solution is to compute more general characteristics (e.g. frequency) and use those instead to extract features. To address this last part, `niimpy` includes the function `extract_features_audio` to clean, downsample, and extract features from audio snippets that have been already anonymized.

Audio dataframes should have the following columns (column names can be different, but in that case they must be provided as parameters):
- `user`: Subject ID
- `device`: Device ID
- `is_silent`: Boolean value, indicates when audio is too quiet to record
- `frequency`: Audio frequency in Hz
- `decibels`: Audio volume in decibels

Niimpy extracts the following audio features:
- `audio_count_silent`: number of times when there has been some sound in the environment
- `audio_count_speech`: number of times when there has been some sound in the environment that matches the range of human speech frequency (65 - 255Hz)
- `audio_count_loud`: number of times when there has been some sound in the environment above 70dB
- `audio_min_freq`: minimum frequency of the recorded audio snippets
- `audio_max_freq`: maximum frequency of the recorded audio snippets
- `audio_mean_freq`: mean frequency of the recorded audio snippets
- `audio_median_freq`: median frequency of the recorded audio snippets
- `audio_std_freq`: standard deviation of the frequency of the recorded audio snippets
- `audio_min_db`: minimum decibels of the recorded audio snippets
- `audio_max_db`: maximum decibels of the recorded audio snippets
- `audio_mean_db`: mean decibels of the recorded audio snippets
- `audio_median_db`: median decibels of the recorded audio snippets
- `audio_std_db`: standard deviations of the recorded audio snippets decibels

In the following, we will analyze audio snippets provided by `niimpy` as an example to illustrate the use of niimpy's audio preprocessing functions.

## 2. Read data

Let's start by reading the example data provided in `niimpy`. These data have already been shaped in a format that meets the requirements of the data schema. Let's start by importing the needed modules. Firstly we will import the `niimpy` package and then we will import the module we will use (audio) and give it a short name for use convinience. 

In [1]:
import niimpy
from niimpy import config
import niimpy.preprocessing.audio as au
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

/u/24/rantahj1/unix/miniconda3/envs/niimpy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now let's read the example data provided in `niimpy`. The example data is in `csv` format, so we need to use the `read_csv` function. When reading the data, we can specify the timezone where the data was collected. This will help us handle daylight saving times easier. We can specify the timezone with the argument **tz**. The output is a dataframe. We can also check the number of rows and columns in the dataframe.

In [2]:
data = niimpy.read_csv(config.MULTIUSER_AWARE_AUDIO_PATH, tz='Europe/Helsinki')
data.shape

(33, 7)

The data was succesfully read. We can see that there are 33 datapoints with 7 columns in the dataset. However, we do not know yet what the data really looks like, so let's have a quick look:

In [3]:
data.head()

,user,device,time,is_silent,double_decibels,double_frequency,datetime
2020-01-09 02:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578528e+09,0,84,4935,2020-01-09 02:08:03.895999908+02:00
2020-01-09 02:38:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578530e+09,0,89,8734,2020-01-09 02:38:03.895999908+02:00
2020-01-09 03:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578532e+09,0,99,1710,2020-01-09 03:08:03.895999908+02:00
2020-01-09 03:38:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578534e+09,0,77,9054,2020-01-09 03:38:03.895999908+02:00
2020-01-09 04:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578536e+09,0,80,12265,2020-01-09 04:08:03.895999908+02:00


In [4]:
data.tail()

,user,device,time,is_silent,double_decibels,double_frequency,datetime
2019-08-13 15:02:17.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565698e+09,1,44,2914,2019-08-13 15:02:17.657999992+03:00
2019-08-13 15:28:59.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565699e+09,1,49,7195,2019-08-13 15:28:59.657999992+03:00
2019-08-13 15:59:01.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565701e+09,0,55,91,2019-08-13 15:59:01.657999992+03:00
2019-08-13 16:29:03.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565703e+09,0,76,3853,2019-08-13 16:29:03.657999992+03:00
2019-08-13 16:59:05.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565705e+09,0,84,7419,2019-08-13 16:59:05.657999992+03:00


By exploring the head and tail of the dataframe we can form an idea of its entirety. From the data, we can see that:

- rows are observations, indexed by timestamps, i.e. each row represents a snippet that has been recorded at a given time and date
- columns are characteristics for each observation, for example, the user whose data we are analyzing
- there are at least two different users in the dataframe
- there are two main columns: `decibels` and `frequency`.

In fact, we can check the first three elements for each user

In [5]:
data.drop_duplicates(['user','time']).groupby('user').head(3)

,user,device,time,is_silent,double_decibels,double_frequency,datetime
2020-01-09 02:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578528e+09,0,84,4935,2020-01-09 02:08:03.895999908+02:00
2020-01-09 02:38:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578530e+09,0,89,8734,2020-01-09 02:38:03.895999908+02:00
2020-01-09 03:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578532e+09,0,99,1710,2020-01-09 03:08:03.895999908+02:00
2019-08-13 07:28:27.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565671e+09,0,51,7735,2019-08-13 07:28:27.657999992+03:00
2019-08-13 07:58:29.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565672e+09,0,90,13609,2019-08-13 07:58:29.657999992+03:00
2019-08-13 08:28:31.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565674e+09,0,81,7690,2019-08-13 08:28:31.657999992+03:00


Sometimes the data may come in a disordered manner, so just to make sure, let's order the dataframe and compare the results. We will use the columns "user" and "datetime" since we would like to order the information according to firstly, participants, and then, by time in order of happening. Luckily, in our dataframe, the index and datetime are the same.

In [6]:
data.sort_values(by=['user', 'datetime'], inplace=True)
data.drop_duplicates(['user','time']).groupby('user').head(3)

,user,device,time,is_silent,double_decibels,double_frequency,datetime
2019-08-13 07:28:27.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565671e+09,0,51,7735,2019-08-13 07:28:27.657999992+03:00
2019-08-13 07:58:29.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565672e+09,0,90,13609,2019-08-13 07:58:29.657999992+03:00
2019-08-13 08:28:31.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565674e+09,0,81,7690,2019-08-13 08:28:31.657999992+03:00
2020-01-09 02:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578528e+09,0,84,4935,2020-01-09 02:08:03.895999908+02:00
2020-01-09 02:38:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578530e+09,0,89,8734,2020-01-09 02:38:03.895999908+02:00
2020-01-09 03:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578532e+09,0,99,1710,2020-01-09 03:08:03.895999908+02:00


The main column names in our dataframe do not match the Niimpy schema. We could provide these column names as parameters but it easier to rename them here.

In [7]:
data = data.rename(columns={'decibels': 'decibels', 'frequency': 'frequency'})
data.head()

,user,device,time,is_silent,double_decibels,double_frequency,datetime
2019-08-13 07:28:27.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565671e+09,0,51,7735,2019-08-13 07:28:27.657999992+03:00
2019-08-13 07:58:29.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565672e+09,0,90,13609,2019-08-13 07:58:29.657999992+03:00
2019-08-13 08:28:31.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565674e+09,0,81,7690,2019-08-13 08:28:31.657999992+03:00
2019-08-13 08:58:33.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565676e+09,0,58,8347,2019-08-13 08:58:33.657999992+03:00
2019-08-13 09:28:35.657999992+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565678e+09,1,36,13592,2019-08-13 09:28:35.657999992+03:00


Ok, it seems like our dataframe was in order. We can start extracting features. However, we need to understand the data format requirements first.

## * TIP! Data format requirements (or what should our data look like)

Data can take other shapes and formats. However, the `niimpy` data schema requires it to be in a certain shape. This means the dataframe needs to have at least the following characteristics:
1. One row per call. Each row should store information about one call only
2. Each row's index should be a timestamp
3. The following five columns are required: 
    - index: date and time when the event happened (timestamp)
    - user: stores the user name whose data is analyzed. Each user should have a unique name or hash (i.e. one hash for each unique user)
    - is_silent: stores whether the decibel level is above a set threshold (usually 50dB)
    - decibels: stores the decibels of the recorded snippet
    - frequency: the frequency of the recorded snippet in Hz
    - NOTE: most of our audio examples come from data recorded with the Aware Framework, if you want to know more about the frequency and decibels, please read https://github.com/denzilferreira/com.aware.plugin.ambient_noise
4. Additional columns are allowed.
5. The names of the columns do not need to be exactly "user", "is_silent", "decibels" or "frequency" as we can pass our own names in an argument.



Column names in our data do not match the Niimpy schema. We could provide these column names as parameters to niimpy functions, but it is simpler to rename them here.

In [8]:
data = data.rename(columns={'double_decibels': 'decibels', 'double_frequency': 'frequency'})

Below is an example of a dataframe that complies with these minimum requirements

In [9]:
example_dataschema = data[['user','is_silent','decibels','frequency']]
example_dataschema.head(3)

,user,is_silent,decibels,frequency
2019-08-13 07:28:27.657999992+03:00,iGyXetHE3S8u,0,51,7735
2019-08-13 07:58:29.657999992+03:00,iGyXetHE3S8u,0,90,13609
2019-08-13 08:28:31.657999992+03:00,iGyXetHE3S8u,0,81,7690


## 4. Extracting features
There are two ways to extract features. We could use each function separately or we could use `niimpy`'s ready-made wrapper. Both ways will require us to specify arguments to pass to the functions/wrapper in order to customize the way the functions work. These arguments are specified in dictionaries. Let's first understand how to extract features using stand-alone functions.

### 4.1 Extract features using stand-alone functions
We can use `niimpy`'s functions to compute communication features. Each function will require two inputs:
- (mandatory) dataframe that must comply with the minimum requirements (see '* TIP! Data requirements above)
- (optional) arguments for stand-alone functions

#### 4.1.1 The argument dictionary for stand-alone functions (or how we specify the way a function works)
We can input two types of arguments to customize the way a stand-alone function works:
- the name of the columns to be preprocessed: Since the dataframe may have different columns, we need to specify which column has the data we would like to be preprocessed. To do so, we can simply pass the name of the column to the argument `audio_column_name`. 

- the way we resample: resampling options are specified in `niimpy` as a dictionary. `niimpy`'s resampling and aggregating relies on `pandas.DataFrame.resample`, so mastering the use of this pandas function will help us greatly in `niimpy`'s preprocessing. Please familiarize yourself with the pandas resample function before continuing. 
    Briefly, to use the `pandas.DataFrame.resample` function, we need a rule. This rule states the intervals we would like to use to resample our data (e.g., 15-seconds, 30-minutes, 1-hour). Neverthless, we can input more details into the function to specify the exact sampling we would like. For example, we could use the *close* argument if we would like to specify which side of the interval is closed, or we could use the *offset* argument if we would like to start our binning with an offset, etc. There are plenty of options to use this command, so we strongly recommend having `pandas.DataFrame.resample` documentation at hand. All features for the `pandas.DataFrame.resample` will be specified in a dictionary where keys are the arguments' names for the `pandas.DataFrame.resample`, and the dictionary's values are the values for each of these selected arguments. This dictionary will be passed as a value to the key `resample_args` in `niimpy`.

Let's see some examples of these parameters:

```python
au.audio_count_loud(data, audio_column_name = "frequency", resample_args = {"rule":"1D"})
au.audio_count_loud(data, audio_column_name = "random_name", resample_args = {"rule":"30min"})
au.audio_count_loud(data, audio_column_name = "other_name", resample_args = {"rule":"45T","origin":"end"})
```

Here, we have three basic feature dictionaries. 

- The first example will analyze the data stored in the column `frequency` in our dataframe. The data will be binned in one day periods
- The second example will analyze the data stored in the column `random_name` in our dataframe. The data will be aggregated in 30-minutes bins
- The third example will analyze the data stored in the column `other_name` in our dataframe. The data will be binned in 45-minutes bins, but the binning will start from the last timestamp in the dataframe. 

**Default values:** if no arguments are passed, `niimpy`'s will aggregate the data in 30-min bins, and will select the audio_column_name according to the most suitable column. For example, if we are computing the minimum frequency, `niimpy` will select *frequency* as the column name. 

#### 4.1.2 Using the functions
Now that we understand how the functions are customized, it is time we compute our first audio feature. Suppose that we are interested in extracting the total number of times our recordings were loud every 50 minutes. We will need `niimpy`'s `audio_count_loud` function.

In [10]:
my_loud_times = au.audio_count_loud(
    data,
    audio_column_name = "decibels",
    resample_args = {"rule":"50T"}
)

Let's look at some values for one of the subjects.

In [11]:
my_loud_times[my_loud_times["user"]=="jd9INuQ5BBlW"]

,user,audio_count_loud,device
2020-01-09 01:40:00+02:00,jd9INuQ5BBlW,1,3p83yASkOb_B
2020-01-09 02:30:00+02:00,jd9INuQ5BBlW,2,3p83yASkOb_B
2020-01-09 03:20:00+02:00,jd9INuQ5BBlW,2,3p83yASkOb_B
2020-01-09 04:10:00+02:00,jd9INuQ5BBlW,0,3p83yASkOb_B
2020-01-09 05:00:00+02:00,jd9INuQ5BBlW,1,3p83yASkOb_B
2020-01-09 05:50:00+02:00,jd9INuQ5BBlW,1,3p83yASkOb_B
2020-01-09 06:40:00+02:00,jd9INuQ5BBlW,1,OWd1Uau8POix
2020-01-09 07:30:00+02:00,jd9INuQ5BBlW,0,OWd1Uau8POix
2020-01-09 08:20:00+02:00,jd9INuQ5BBlW,1,OWd1Uau8POix
2020-01-09 09:10:00+02:00,jd9INuQ5BBlW,1,OWd1Uau8POix


Let's remember how the original data looks like for this subject

In [12]:
data[data["user"]=="jd9INuQ5BBlW"].head(7)

,user,device,time,is_silent,decibels,frequency,datetime
2020-01-09 02:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578528e+09,0,84,4935,2020-01-09 02:08:03.895999908+02:00
2020-01-09 02:38:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578530e+09,0,89,8734,2020-01-09 02:38:03.895999908+02:00
2020-01-09 03:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578532e+09,0,99,1710,2020-01-09 03:08:03.895999908+02:00
2020-01-09 03:38:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578534e+09,0,77,9054,2020-01-09 03:38:03.895999908+02:00
2020-01-09 04:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578536e+09,0,80,12265,2020-01-09 04:08:03.895999908+02:00
2020-01-09 04:38:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578537e+09,0,52,7281,2020-01-09 04:38:03.895999908+02:00
2020-01-09 05:08:03.895999908+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578539e+09,0,63,14408,2020-01-09 05:08:03.895999908+02:00


We see that the bins are indeed 50-minutes bins, however, they are adjusted to fixed, predetermined intervals, i.e. the bin does not start on the time of the first datapoint. Instead, `pandas` starts the binning at 00:00:00 of everyday and counts 50-minutes intervals from there. 

If we want the binning to start from the first datapoint in our dataset, we need the origin parameter and a for loop.

In [13]:
users = list(data['user'].unique())
results = []
for user in users:
    start_time = data[data["user"]==user].index.min()
    results.append(au.audio_count_loud(
        data[data["user"]==user],
        audio_column_name="decibels",
        resample_args={"rule":"50T"}
    ))
my_loud_times = pd.concat(results)

In [14]:
my_loud_times

,user,audio_count_loud,device
2019-08-13 07:30:00+03:00,iGyXetHE3S8u,1,Cq9vueHh3zVs
2019-08-13 08:20:00+03:00,iGyXetHE3S8u,1,Cq9vueHh3zVs
2019-08-13 09:10:00+03:00,iGyXetHE3S8u,1,Cq9vueHh3zVs
2019-08-13 10:00:00+03:00,iGyXetHE3S8u,1,Cq9vueHh3zVs
2019-08-13 10:50:00+03:00,iGyXetHE3S8u,2,Cq9vueHh3zVs
2019-08-13 11:40:00+03:00,iGyXetHE3S8u,1,Cq9vueHh3zVs
2019-08-13 12:30:00+03:00,iGyXetHE3S8u,0,Cq9vueHh3zVs
2019-08-13 13:20:00+03:00,iGyXetHE3S8u,0,Cq9vueHh3zVs
2019-08-13 14:10:00+03:00,iGyXetHE3S8u,1,Cq9vueHh3zVs
2019-08-13 15:00:00+03:00,iGyXetHE3S8u,0,Cq9vueHh3zVs


### 4.2 Extract features using the wrapper
We can use `niimpy`'s ready-made wrapper to extract one or several features at the same time. The wrapper will require two inputs:
- (mandatory) dataframe that must comply with the minimum requirements (see '* TIP! Data requirements above)
- (optional) an argument dictionary for wrapper

#### 4.2.1 The argument dictionary for wrapper (or how we specify the way the wrapper works)
The argument dictionary contains the arguments for each stand-alone function we would like to employ. Its keys are the feature functions we want to compute. Its values are argument dictionaries created for each stand-alone function we will employ. 
Let's see some examples of wrapper dictionaries:

In [15]:
wrapper_features1 = {au.audio_count_loud:{"audio_column_name":"decibels","resample_args":{"rule":"1D"}},
                     au.audio_max_freq:{"audio_column_name":"frequency","resample_args":{"rule":"1D"}}}

- `wrapper_features1` will be used to analyze two features, `audio_count_loud` and `audio_max_freq`. For the feature audio_count_loud, we will use the data stored in the column `decibels` in our dataframe and the data will be binned in one day periods.  For the feature audio_max_freq, we will use the data stored in the column `frequency` in our dataframe and the data will be binned in one day periods. 

In [16]:
wrapper_features2 = {au.audio_mean_db:{"audio_column_name":"random_name","resample_args":{"rule":"1D"}},
                     au.audio_count_speech:{"audio_column_name":"decibels", "audio_freq_name":"frequency", "resample_args":{"rule":"5H","offset":"5min"}}}

- `wrapper_features2` will be used to analyze two features, `audio_mean_db` and `audio_count_speech`. For the feature audio_mean_db, we will use the data stored in the column `random_name` in our dataframe and the data will be binned in one day periods.  For the feature audio_count_speech, we will use the data stored in the column `decibels` in our dataframe and the data will be binned in 5-hour periods with a 5-minute offset. Note that for this feature we will also need another column named "audio_freq_column", this is because the speech is not only defined by the amplitude of the recording, but the frequency range. 

In [17]:
wrapper_features3 = {au.audio_mean_db:{"audio_column_name":"one_name","resample_args":{"rule":"1D","offset":"5min"}},
                     au.audio_min_freq:{"audio_column_name":"one_name","resample_args":{"rule":"5H"}},
                     au.audio_count_silent:{"audio_column_name":"another_name","resample_args":{"rule":"30T","origin":"end_day"}}}

- `wrapper_features3` will be used to analyze three features, `audio_mean_db`, `audio_min_freq`, and `audio_count_silent`. For the feature audio_mean_db, we will use the data stored in the column `one_name` and the data will be binned in one day periods with a 5-min offset.  For the feature audio_min_freq, we will use the data stored in the column `one_name` in our dataframe and the data will be binned in 5-hour periods. Finally, for the feature audio_count_silent, we will use the data stored in the column `another_name` in our dataframe and the data will be binned in 30-minute periods and the origin of the bins will be the ceiling midnight of the last day.

**Default values:** if no arguments are passed, `niimpy`'s default values are either "decibels", "frequency", or "is_silent" for the communication_column_name, and 30-min aggregation bins. The column name depends on the function to be called. Moreover, the wrapper will compute all the available functions in absence of the argument dictionary. 

#### 4.2.2 Using the wrapper
Now that we understand how the wrapper is customized, it is time we compute our first communication feature using the wrapper. Suppose that we are interested in extracting the audio_count_loud  duration every 50 minutes. We will need `niimpy`'s `extract_features_audio` function, the data, and we will also need to create a dictionary to customize our function. Let's create the dictionary first

In [18]:
wrapper_features1 = {au.audio_count_loud:{"audio_column_name":"decibels","resample_args":{"rule":"50T"}}}

Now, let's use the wrapper

In [19]:
results_wrapper = au.extract_features_audio(data, features=wrapper_features1)
results_wrapper.head(5)

,user,device,audio_count_loud
2019-08-13 07:30:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1
2019-08-13 08:20:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1
2019-08-13 09:10:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1
2019-08-13 10:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1
2019-08-13 10:50:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,2


Our first attempt was succesful. Now, let's try something more. Let's assume we want to compute the audio_count_loud and audio_min_freq in 1-hour bins.

In [20]:
wrapper_features2 = {au.audio_count_loud:{"audio_column_name":"decibels","resample_args":{"rule":"1H"}},
                     au.audio_min_freq:{"audio_column_name":"frequency", "resample_args":{"rule":"1H"}}}
results_wrapper = au.extract_features_audio(data, features=wrapper_features2)
results_wrapper.head(5)

,user,device,audio_count_loud,audio_min_freq
2019-08-13 07:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1,7735.0
2019-08-13 08:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1,7690.0
2019-08-13 09:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1,756.0
2019-08-13 10:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,2,3059.0
2019-08-13 11:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,2,12278.0


Great! Another successful attempt. We see from the results that more columns were added with the required calculations. This is how the wrapper works when all features are computed with the same bins. Now, let's see how the wrapper performs when each function has different binning requirements. Let's assume we need to compute the audio_count_loud every day, and the audio_min_freq every 5 hours with an offset of 5 minutes.

In [21]:
wrapper_features3 = {au.audio_count_loud:{"audio_column_name":"decibels","resample_args":{"rule":"1D"}},
                     au.audio_min_freq:{"audio_column_name":"frequency", "resample_args":{"rule":"5H", "offset":"5min"}}}
results_wrapper = au.extract_features_audio(data, features=wrapper_features3)
results_wrapper.head(5)

,user,device,audio_count_loud,audio_min_freq
2019-08-13 00:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,10.0,NaN
2020-01-09 00:00:00+02:00,jd9INuQ5BBlW,3p83yASkOb_B,7.0,NaN
2020-01-09 00:00:00+02:00,jd9INuQ5BBlW,OWd1Uau8POix,5.0,NaN
2019-08-13 05:05:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,NaN,756.0
2019-08-13 10:05:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,NaN,2914.0


The output is once again a dataframe. In this case, two aggregations are shown. The first one is the daily aggregation computed for the `audio_count_loud` feature. The second one is the 5-hour aggregation period with 5-min offset for the `audio_min_freq`. We must note that because the `audio_min_freq`feature is not required to be aggregated daily, the daily aggregation timestamps have a NaN value. Similarly, because the `audio_count_loud`is not required to be aggregated in 5-hour windows, its values are NaN for all subjects. 

#### 4.2.3 Wrapper and its default option
The default option will compute all features in 30-minute aggregation windows. To use the `extract_features_audio` function with its default options, simply call the function. 

In [22]:
default = au.extract_features_audio(data, features=None)

In [23]:
default.head()

,user,device,audio_count_silent,audio_count_speech,audio_count_loud,audio_min_freq,audio_max_freq,audio_mean_freq,audio_median_freq,audio_std_freq,audio_min_db,audio_max_db,audio_mean_db,audio_median_db,audio_std_db
2019-08-13 07:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,0,NaN,NaN,7735.0,7735.0,7735.0,7735.0,NaN,51.0,51.0,51.0,51.0,NaN
2019-08-13 07:30:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,0,NaN,1.0,13609.0,13609.0,13609.0,13609.0,NaN,90.0,90.0,90.0,90.0,NaN
2019-08-13 08:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,0,NaN,1.0,7690.0,7690.0,7690.0,7690.0,NaN,81.0,81.0,81.0,81.0,NaN
2019-08-13 08:30:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,0,NaN,0.0,8347.0,8347.0,8347.0,8347.0,NaN,58.0,58.0,58.0,58.0,NaN
2019-08-13 09:00:00+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1,NaN,0.0,13592.0,13592.0,13592.0,13592.0,NaN,36.0,36.0,36.0,36.0,NaN


## 5. Implementing own features

If none of the provided functions suits well, We can implement our own customized features easily. To do so, we need to define a function that accepts a dataframe and returns a dataframe. The returned object should be indexed by user and timestamps (multiindex).
Let's assume we need a new function that counts sums all frequencies. Let's first define the function

In [24]:
def audio_sum_freq(df, audio_column_name = "frequency", resample_args= {"rule":"30T"}):
    if len(df)>0:
        result = df.groupby('user')[audio_column_name].resample(**resample_args).sum()
        result = result.to_frame(name='audio_sum_freq')
        result = result.reset_index("user")
        result.index.rename("datetime", inplace=True)
        return result
    return None

Then, we can call our new function in the stand-alone way or using the `extract_features_audio` function. Alternatively, we can pass the feature function to the wrapper. Let's read again the data and assume we want the default behavior of the wrapper. 

In [25]:
customized_features = au.extract_features_audio(data, features={audio_sum_freq: {}})

In [26]:
customized_features.head()

,user,audio_sum_freq
datetime,,
2019-08-13 07:00:00+03:00,iGyXetHE3S8u,7735
2019-08-13 07:30:00+03:00,iGyXetHE3S8u,13609
2019-08-13 08:00:00+03:00,iGyXetHE3S8u,7690
2019-08-13 08:30:00+03:00,iGyXetHE3S8u,8347
2019-08-13 09:00:00+03:00,iGyXetHE3S8u,13592
